In [1]:
from db import fetch
from tqdm import tqdm

In [118]:
query = """
SELECT id, __updatetime, type, winner, loser, status

FROM contests

WHERE type = 'H' and status = 'completed'

LIMIT 10000
"""

In [119]:
df = fetch(query)
df.shape

(10000, 6)

In [120]:
df.head()

,id,__updatetime,type,winner,loser,status
0,10003088776399,2019-03-09 11:20:52.349,H,6.397517e+15,1.677400e+15,completed
1,1000607187102953,2019-03-08 11:20:41.036,H,7.215143e+15,4.486515e+15,completed
2,100070134290293,2019-03-08 11:15:44.029,H,5.188046e+15,4.936061e+15,completed
3,1000750612771762,2019-03-08 11:20:41.036,H,8.922595e+15,2.854242e+15,completed
4,1000839704742657,2019-03-08 11:20:41.036,H,6.518224e+15,6.638050e+15,completed


In [121]:
lookup = {}

for _, row in tqdm(df.iterrows()):
    w = row['winner'] 
    l = row['loser']
    t = row['__updatetime']
    
    if w in lookup.keys():
        lookup[w].append((t, "W"))
    else:
        lookup[w] = [(t, "W")]
    
    if l in lookup.keys():
        lookup[l].append((t, "L"))
    else:
        lookup[l] = [(t, "L")]

10000it [00:01, 6435.18it/s]


In [122]:
query = """
select userid, amt, __updatetime from transactions

WHERE contest_type = 'contest' and type = 'S'

LIMIT 10000
"""

In [123]:
df = fetch(query)
df.shape

(10000, 3)

In [124]:
df.head()

,userid,amt,__updatetime
0,2571710851091570,20.0,2019-03-05 11:13:39.968
1,6527839797482430,1.0,2019-02-26 11:09:37.169
2,1836563405302799,1.0,2019-01-11 21:40:59.522
3,1122443791606407,5.0,2019-01-11 21:40:59.522
4,3563729386247051,2.5,2019-01-11 21:40:59.522


In [125]:
users_with_deposit = []

for _, row in tqdm(df.iterrows()):
    u = row['userid']
    a = row['amt']
    t = row['__updatetime']
    
    if u in lookup.keys():
        users_with_deposit.append(u)
        lookup[u].append((t, a))

10000it [00:01, 9488.54it/s]


In [126]:
# sort games by timestamp
for k, v in lookup.items():
    lookup[k] = sorted(v, key=lambda x: x[0])

In [127]:
# remove timespace
for k, v in lookup.items():
    lookup[k] = list(map(lambda x: x[1], v))

In [137]:
for u in users_with_deposit:
    record = lookup[u]
    if (1.0 in record):
        if check(record) and len(record) < 6:
            print(record)

[4.0, 'W', 1.0]
[1.0, 'L', 'W', 'L', 2.5]
[1.0, 'L', 'W', 'L', 2.5]
[10.0, 'W', 'W', 1.0]
[1.0, 'L', 'L', 'W', 1.0]
[1.0, 'L', 'L', 10.0]
[5.0, 'L', 'W', 'W', 1.0]
[1.0, 'L', 'L', 1.0]
[10.0, 'W', 'W', 1.0]
[1.0, 'L', 'L', 1.0]
[1.0, 'L', 'L', 'W', 1.0]
[1.0, 'L', 'L', 10.0]
[4.0, 'W', 1.0]
[5.0, 'L', 'W', 'W', 1.0]


In [136]:
def check(seq):
    desired_sequence = [float, str, float]
    
    for s in seq:
        if len(desired_sequence) == 0:
            return True
        
        t = desired_sequence[0]
        if isinstance(s, t):
            desired_sequence.pop(0)
    return len(desired_sequence) == 0